## **Welcome to Week Three of BrainTank Deep Learning:**

Lets get started:

---

First thing we are going to do is run this piece of code that will download important files for this weeks challenge. Take a look at:

1.   model.state

In [2]:
!git clone https://github.com/BrainTankDeepLearning/Week3.git

Cloning into 'Week3'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


Helper Functions:
---
Here I have provided two helper functions that will help us use and view some of the data we working with. You DO NOT need to edit any of these, but feel free to play around with them.

In [3]:
import torch
from torch.nn import Module
import torchvision
import numpy as np
import csv
import matplotlib.pyplot as plt
import os
import pandas as pd
import math
import sys

random_seed = 3
torch.manual_seed(random_seed)

import random
random.seed(0)

def get_datasets(batch_size = 9):
  train_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('/files/', train=True, download=True,
                              transform=torchvision.transforms.Compose([
                                torchvision.transforms.ToTensor(),
                                torchvision.transforms.Normalize(
                                  (0.1307,), (0.3081,))
                              ])),
    batch_size=batch_size, shuffle=False, drop_last = True)

  test_loader = torch.utils.data.DataLoader(
    torchvision.datasets.MNIST('/files/', train=False, download=True,
                              transform=torchvision.transforms.Compose([
                                torchvision.transforms.ToTensor(),
                                torchvision.transforms.Normalize(
                                  (0.1307,), (0.3081,))
                              ])),
    batch_size=batch_size, shuffle=False, drop_last = True)
  

  
  return train_loader, test_loader

def print_image(image, target = None, prediction = None):
  # settings
  h, w = 26, 26        # for raster image
  nrows, ncols = 3, 3  # array of sub-plots
  figsize = [6, 8]     # figure size, inches

  # create figure (fig), and array of axes (ax)
  fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)

  # plot simple raster image on each sub-plot
  for i, axi in enumerate(ax.flat):
    if i >= nrows * ncols:
      break
    # i runs from 0 to (nrows*ncols-1)
    # axi is equivalent with ax[rowid][colid]
    img = image[i]
    axi.imshow(img.squeeze(), alpha=0.25)
    # get indices of row/column
    rowid = i // ncols
    colid = i % ncols
    # write row/col indices as axes' title for identification
    if prediction is None and target is not None:
      axi.set_title("Target:"+str(target[i].item()))
    if prediction is not None and target is None:
      axi.set_title("Prediction:" + str(prediction[i].item()))
    if prediction is not None and target is not None:
      axi.set_title("Target:"+str(target[i].item())+" Prediction:" + str(prediction[i].item()))

  plt.tight_layout(True)
  plt.show()

def create_one_hot(labels):
  out = torch.empty(size = (len(labels), 10))
  for i, label in enumerate(labels):
    one_hot = torch.zeros(10)
    one_hot[label.item()] = 1
    out[i] = one_hot
  return out

def softmax_loss(prediction, one_hot):
  prediction = torch.log(prediction)

  target = torch.empty(size = (len(one_hot), ), dtype = torch.long)
  for i, row in enumerate(one_hot):
    index = (row == 1).nonzero(as_tuple=True)[0]
    target[i] = index.item()

  loss = torch.nn.functional.nll_loss(prediction, target)

  return loss

# Our Tasks:

---

**1.   Create our own Neural Network Layer class**

Helpful Formula for calculate a forward propagation:

*   out = in * A^t + b

**2.   Create a neural network that we can use to predict the label associated with a image's pixel values**

*   Our model will take in a batch of images as inputs and output vector of length 10 with percent values associated with the liklihood it thinks a picture belongs to it's corresponding label

**3.   Devize a strategy to test our network and see if it is doin a good or bad job prediciting labels**

**4. Save and load our model to prevent needing to retrain the model every time we want to test**

In [76]:
class NeuralNetworkLayer(torch.nn.Module):
  def __init__(self, input_layers : int, output_layers : int):
    super(NeuralNetworkLayer, self).__init__()
    
    self.A = torch.empty(size = (output_layers, input_layers))
    self.b = torch.empty(size = (output_layers, ))

    self.A = torch.randn_like(self.A)
    self.b = torch.randn_like(self.b)

    self.A = torch.nn.Parameter(self.A)
    self.b = torch.nn.Parameter(self.b)

  def forward(self, input_activation):
    A_transposed = torch.transpose(self.A, 0, 1)
    output_activation = torch.matmul(input_activation, A_transposed)
    output_activation_with_bias = output_activation + self.b

    return output_activation_with_bias

class ScoreClassifier(torch.nn.Module):
  def __init__(self):
    super(ScoreClassifier, self).__init__()
    
    self.NN_layer1 = NeuralNetworkLayer(784, 400)
    self.NN_layer2 = NeuralNetworkLayer(400, 200)
    self.NN_layer3 = NeuralNetworkLayer(200, 100)
    self.NN_layer4 = NeuralNetworkLayer(100, 10)

  def forward(self, image):
    image = image.squeeze(dim = 1)
    image = image.view(9, 784)

    out = self.NN_layer1(image)
    out = torch.sigmoid(out)
    out = self.NN_layer2(out)
    out = torch.sigmoid(out)
    out = self.NN_layer3(out)
    out = torch.sigmoid(out)
    out = self.NN_layer4(out)

    out = torch.nn.functional.softmax(out, dim = 1)

    return out

def train(model, train_dataset, optim):
  for epoch_number in range(5):
    avg_loss = 0
    for data in train_ds:
      optim.zero_grad()

      images = data[0]
      labels = data[1]

      prediction = model(images)
      ground_truths = create_one_hot(labels)

      loss = softmax_loss(prediction, ground_truths)

      loss.backward()
      optim.step()

      avg_loss += loss.item()
    avg_loss /= (len(train_ds))
    print(f"loss: {avg_loss}")
  
  torch.save(model.state_dict(), "model.state")

def test(model, test_dataset):
  total_correct = 0
  for data in test_dataset:
    images = data[0]
    labels = data[1]

    prediction = model(images)

    max_indexes = torch.argmax(prediction, axis = 1)

    n_correct = (max_indexes == labels)
    n_correct = torch.sum(n_correct == True).item()

    total_correct += n_correct

    #print_image(images, target = labels, prediction = max_indexes)

  print("Accuracy:", total_correct / len(test_dataset) / 9)

if __name__ == "__main__":
  # load the datasets:
  train_ds, test_ds = get_datasets(batch_size = 9)

  # define the model
  model = ScoreClassifier()

  if os.path.isfile("Week3/model.state"):
    model.load_state_dict(torch.load("Week3/model.state"))
    print("Loaded model from: model.state")

  # optimizer
  optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

  # train
  train(model, train_ds, optimizer)

  # test
  test(model, test_ds)


Loaded model from: model.state
Accuracy: 0.963096309630963
